In [ ]:
import ee
import geemap
import numpy as np
import folium
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project='mini-mapper')
print(ee.String('Hello from the Earth Engine servers!').getInfo())


In [ ]:
# TODO: these variables will be set from the user input passed through from the Rust backend

geojson_object = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -3.283555,
        51.512079
      ],
      [
        -3.098898,
        51.513361
      ],
      [
        -3.096152,
        51.444054
      ],
      [
        -3.269139,
        51.441913
      ],
      [
        -3.283555,
        51.512079
      ]
    ]
  ]
}

# AOI polygon
extent = ee.Geometry(geojson_object)

# date range
start_date = ee.Date.fromYMD(2024,1,1)
end_date = ee.Date.fromYMD(2024,6,30)

# sensor type (S2)
sensor_collection = "COPERNICUS/S2_SR_HARMONIZED"

# name of bands (depends on analysis type?)

# cloud mask percentage
cloud_mask_percentage = 20

# initalize map
map = geemap.Map()

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
pallete = {"min":0, "max":1, 'palette':color}

In [ ]:
# accessing the data with the above filters
imagery = ee.ImageCollection(sensor_collection) \
    .filterBounds(extent) \
    .filterDate(start_date, end_date) \
    .select('B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_mask_percentage))

# TODO: how to make the bands to select a variable as they may depend on future sensor options

In [ ]:
# metadata: number of images
# display(imagery.size())

# # metadata: full json of the unprocessed image collection
# print(imagery)

# # metadata: get the exact dates of the retrieved images used for the average
# dates = imagery.aggregate_array('system:time_start').getInfo()
# dates = [np.datetime64(ee.Date(date).format('YYYY-MM-dd').getInfo()) for date in dates]
# print(f'Dates: {dates}')

# TODO: combine into one metadata object to be passed back through to the frontend

In [ ]:
# NDVI calculation function (B8 = NIR, B4 = Red)
def calculate_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return img.addBands(ndvi)

# average the data, apply the NDVI calculation and clip to extent
imagery_avg = imagery.median()
imagery_avg_ndvi = calculate_ndvi(imagery_avg).select('NDVI')
clipped_avg_ndvi = imagery_avg_ndvi.clip(extent)

# define parameters for visualising NDVI
ndvi_params = {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}

In [ ]:
# visualising with folium
map_center = extent.centroid().coordinates().getInfo()[::-1]  # center the map on the extent
folium_map = folium.Map(location=map_center, zoom_start=10)

# add the layer to the map
folium.TileLayer(
    tiles=clipped_avg_ndvi.getMapId(ndvi_params)['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='NDVI'
).add_to(folium_map)

folium_map

In [ ]:
# visual inspection
map.centerObject(extent)
map.addLayer(clipped_avg_ndvi, ndvi_params, 'NDVI')
map